In [226]:
import pandas as pd
import numpy as np
from typing import *
import re
from three_answer import Answer
from functools import reduce
import operator

In [227]:
content: str = ""
read_content: str = ""
with open("output/three_output1.txt", "r") as file:
    content=[ctnt.replace("*", "").strip() for ctnt in file.readlines()]

# with open("output/three_output.txt", "r") as file:
#     read_content=file.read()

INTERTIAL_vocabulary = ["sure", "I'd be happy", "happy", "please", "help", "given sentence", \
                        "I", "let", "based", "here are", "understood", "predictions", "mentioned", \
                            "sentence"]
escape_vocabulary=[re.escape(word) for word in INTERTIAL_vocabulary]
pattern = rf"\b({'|'.join(escape_vocabulary)})\b"

In [228]:
content[:11]

['Understood! Here are my predictions for each element you requested:',
 'Project: Basin',
 'Risk: Extreme weather events',
 'Stakeholder: Local communities and governments  1000',
 '',
 "Of course! I'd be happy to help you identify the keywords in the given sentence. Here are my predictions:",
 'Project: Stream flow augmentation',
 'Risk: Ecological impacts',
 'Stakeholder: Water agency and public',
 'Please let me know if these labels are accurate, or if you would like me to adjust my prediction based on your feedback.  1001',
 '']

In [229]:
def list_split(cent: List[str] = content):
    index_int = 0
    for idx, ele in enumerate(cent):
        if re.search(r"^\d+$|\d$", ele):
            yield content[index_int: idx+1]
            index_int = idx+1
    return content[index_int: ]

In [230]:
res = list(list_split(content))

In [231]:
def digital_distinction(target: str) -> List[str]:
    """
    target here is supposed to be the last string of each response
    """
    match = re.search(pattern=r"\d+$", string=target)
    if not match: return [target] # for three_output.txt
    pos, dig= match.span(), match.group()
    return [target[:pos[0]], dig]

In [232]:
# res = [[*digital_distinction(sub_sec.pop(-1)), *sub_sec] for sub_sec in res]
for idx, sub_sec in enumerate(res):
    res[idx] = [sub_val \
                for val in map(digital_distinction, sub_sec) \
                    for sub_val in val]
# res = [[reduce(operator.concat, digital_distinction(val)) \
#               for val in sub_sec]\
#                      for sub_sec in res ]
# a bit problem, pop indeed prevent existence of extra the same value 
# but it makes the order of string into chaos

In [233]:
def interial_check(tar: Union[str, int])-> Union[str, Union[bool, int]]:
    if not isinstance(tar, str): return (None, False)
    digital = r"\d+$"
    numbers = re.search(pattern=digital, string=tar) # return the ditigal words hidden in the last senntece
    return re.search(pattern=pattern, string=tar, flags=re.IGNORECASE), int(numbers.group()) if numbers else False

In [234]:
def prefix_clean(string):
    if not isinstance(string, str): return string
    pattern = r"\* |\d+\.\s|\`"
    matches = re.sub(pattern=pattern, repl='', string=string, flags=re.MULTILINE)
    return matches

In [235]:
"""
structure of interial_check cannot be changed, better return a stable or single value 
"""
test_res = [[prefix_clean(sub_nele) \
             for sub_nele in nele \
                if not (not sub_nele or interial_check(sub_nele)[0])] \
                    for nele in res]

In [ ]:
# prefix_clean(res[1][-1])
test_res[-10]

In [237]:
def int_geter(tar: List[Union[str, int]]):
    num = map(int, filter(lambda a: a.isdigit(), tar))
    num = next(iter(num))
    print(num, tar)
    tar.pop(tar.index(str(num)))
    return num, tar

In [238]:
res[-2]

['',
 'Understood! Here are the three categories I have identified based on the given sentence:',
 'Project: Climate model output analysis',
 'Risk: Incorrect communication of aridity changes',
 'Stakeholders: Hydroclimatologists, ecohydrologists  ',
 '9896']

In [252]:
template: List[str] = ["stakeholder:", "risk:", "project:", "stakeholders:"] 
head_pattern = r"^(?:{})".format("|".join(template))
def honer_solution(obj: List[str]):
    """
    problem 1: index may not be able always keep the number in the last line
    problem 2: there will be extra explaination sentnece, find them out
    """
    global head_pattern
    three_ele, index = obj[:-1], obj[-1]
    extra = []
    for idx, _ in enumerate(three_ele):
        while idx<len(three_ele) and not re.search(head_pattern, three_ele[idx], re.IGNORECASE):
            category = re.search(head_pattern, three_ele[idx-1], re.IGNORECASE).group(0)
            extra.append([category, three_ele.pop(idx)])

    three_ele = [ab_ele.split(":") for ab_ele in three_ele]
    return {real_time[0]: [*(real_time[1].split(",", -1) if real_time[1] else [])] for real_time in three_ele}, extra, index

In [ ]:
abs_data = []
for nuidx, val in enumerate(test_res):
    try:
        abs_data.append(honer_solution(val))
    except Exception as e:
        print(nuidx)
        # break

# possible worked for index 3740
pattern = fr"\b({'|'.join(template)})\s*[:-]"

In [261]:
test_res[3740]

['Project - Refers to the act of planting trees in an urban area.',
 'Risk - Highlights the potential risks associated with tree planting, such as poor tree survival rates.',
 'Stakeholder - Suggests that various stakeholders may be impacted by tree planting initiatives, including city officials, residents, and environmental groups.  ',
 '6433']

In [243]:
honer_solution(test_res[690]) 

['Risk: None',
 'Project: R.A.V.E. Space',
 'Stakeholders:',
 'Local communities',
 'Inhabitants',
 'Project partners (Interreg IIIB CADSES Programme)  ',
 '1872']

In [140]:
hder, std_col = True, ["Project", "Risk", "Stakeholder", "idx"]
for idv, order_ele in enumerate(abs_data):
    normal, extra, index = order_ele
    if not bool(normal): continue
    normal["idx"] = [index]
    normal = {key: ['|'.join(value).strip()] if isinstance(value, list) else str(value) for key, value in normal.items()}
    fuck_off = pd.DataFrame(normal).rename(columns={"Stakeholders": "Stakeholder"})
    if len(fuck_off) < 4:
        fuck_off = fuck_off.reindex(columns=std_col).fillna("None")
    try: 
        fuck_off.to_csv("Generated_Result/three_output1.csv", index=False, mode="a", header=hder)
        hder = False
    except Exception as e:
        print(idv)

In [142]:
abs_data[821][0]

{'Risks': [], 'Stakeholders': [], 'idx': ['935']}

In [130]:
abs_data[0]

({'Project': [' Electricity generation sector'],
  'Risk': [' Increased emissions through offsets'],
  'Stakeholders': [' Host countries', ' project developers', ' investors'],
  'idx': ['0']},
 [],
 '0')

In [124]:
# odle = abs_data[0][0]
# odle["idx"] = abs_data[0][-1]
# {key: ['|'.join(value).strip()] if isinstance(value, list) else str(value) for key, value in odle.items()}
fuck_off

,Project,Risk,Stakeholder,idx
0,Urban drain restoration project,Cost savings through restoration,Homeowners and local government,959


In [127]:
res[220]
# {real_time[0]: [*(real_time[1].split(", ", -1) if real_time[1] else [])] for real_time in three_ele}, index

['',
 'Understood! Here are my predictions for each element mentioned in the sentence:',
 'Risk: Delayed project completion',
 'This phrase suggests that there might be dangers associated with completing the airport development project within the allotted time frame. These dangers could involve things like poor project management, insufficient funding, or unexpected obstacles during construction. By identifying these risks early on and developing strategies to mitigate them, project managers can reduce the likelihood of delays and ensure timely completion of the project.',
 'Stakeholder: Airport users',
 'These individuals may consist of travel',
 '253']

In [6]:
option2="""Based on the given sentence, I have identified the following risks and their types:
Risks:
Risk 1: Landslide risk (geological risk)
Risk Type: Seismic hazard risk
Risk 2: Seismic risk (geological risk)
Risk Type: Earthquake risk
Risk 3: Structural risk (engineering risk)
Risk Type: Failure of non-structural components risk

Please note that these are my interpretations based on the given sentence, and there may be other potential risks and risk types that could be identified with further analysis."""

option3="""  Of course! I'd be happy to help you identify and summarize the risks in the given sentence.
Risks:
* Terrorist attack
* Structural stability
* Accessibility to terrorists
* Resistance to specific threat

Risk Type:

I have identified four types of risks in the sentence:
1. Terrorist attack: This risk refers to the potential threat of a terrorist attack on the bridge, which could cause significant damage and disrupt the functioning of the bridge.
2. Structural stability: This risk refers to the potential failure of the bridge's structural components, which could lead to a collapse of the bridge and cause harm to people and structures.
3. Accessibility to terrorists: This risk refers to the ease with which terrorists could access the bridge and carry out a attack.
4. Resistance to specific threat: This risk refers to the ability of the bridge's components to withstand a specific type of threat, such as a bomb or a missile attack.
I hope this helps! Let me know if you have any further questions.

"""
test.append(option2)
test.append(option3)


In [7]:
%run each_answer.py
sample1=[]

for val in test:
    proclass=Answer(val)
    result=proclass.parse()
    sample1.append(result)
    print(result)

['0', '', 'Lack of data', 'Complex environmental conditions over time', 'Insufficient risk assessment', 'Inadequate risk control measures', '', '', '', 'Operational risk', 'Environmental risk', 'Data-related risk', 'Time-related risk', '', '0']
['0', '', 'Neglecting maintenance work', 'Insufficient investment in maintenance', 'Growing backlog of maintenance work', 'Identification of maintenance significant units', 'Risk assessment of infrastructure systems', '', '', 'Operational risk', 'Financial risk', 'Reputation risk', 'Compliance risk', '', '0']
['0', '', 'Drou']
['0', '', ' Landslide risk (geological risk)', ' Seismic hazard risk', ' Seismic risk (geological risk)', ' Earthquake risk', ' Structural risk (engineering risk)', ' Failure of non-structural components risk', '', '0']
['0', '', 'Terrorist attack', 'Structural stability', 'Accessibility to terrorists', 'Resistance to specific threat', '', '', '', '0', 'Terrorist attack: This risk refers to the potential threat of a terror

In [8]:
# pattern = r"[^a-zA-Z\s,';-]+"
# for val in sample1[-2]:
#     matches = re.split(pattern, val)
#     print(matches)

In [12]:
%run each_answer.py
for index, nounce in enumerate(test):
    # if index == 4:
    jack = ShortSentenceCut(nounce)
    risk_set : List[str] = jack.input_structure()
    print(risk_set)

(True, 6, 9)
(True, 7, 9)
(False, 3, 0)
(False, 10, 0)
(True, 6, 9)


In [13]:
%run each_answer.py
jack = ShortSentenceCut(test[3])
valid, value = jack.format_tracking()
print(valid, "\n\n", value)

True 

 [{' Structural risk (engineering risk)', ' Seismic risk (geological risk)', ' Landslide risk (geological risk)'}, {' Seismic hazard risk', ' Earthquake risk', ' Failure of non-structural components risk'}]


In [14]:
# %run each_answer.py
read_content=""
for val in ["output/output.txt", "output/output1.txt", "output/output2.txt", "output/output3.txt"]:
    with open(val, "r") as file:
        read_content+=file.read()

all = read_content.split("\n\n\n")

In [78]:
%run each_answer.py
success = []

for index, nounce in enumerate(all[:100]):
    try:
        jack = ShortSentenceCut(nounce)
        risk_set : List[str] = jack.two_group()
        success.append(risk_set)
    except Exception as exp:
        print(index, "\n", exp)

In [77]:
success[0]

{'anomaly': [],
 'normal': {'compose': [('xxx risk (mentioned in the sentence as "there is no reason to consider the model invalid for driver behavior in basic segments of the freeways in Iran, under non-congested conditions.")',
    0),
   ('Uncertain position of vehicles (caused by their acceleration or deceleration)',
    0),
   ('Safety risk (related to the potential for accidents or injuries in emergency situations)',
    0),
   ('Environmental risk (related to the impact of environmental conditions on safety measures)',
    0),
   ('Equipment risk (related to the potential malfunctioning of safety equipment)',
    0),
   ('Emergency preparedness risk (related to the adequacy of emergency measures and training)',
    0),
   ('Human interaction risk (related to the potential for conflicts or inadequate communication among personnel in emergency situations)',
    0)]}}

In [80]:
new_tough = []
for idx, itm in enumerate(success):
    kept = itm['normal']['compose']
    for val in kept:
        val = val + (idx, )
        new_tough.append(val)

In [84]:
val = pd.DataFrame(new_tough, columns=["risk", "types", "groups"])

In [100]:
val.head()

,risk,types,groups
0,Lack of data,Environmental risk,0
1,Complex environmental conditions over time,Operational risk,0
2,Insufficient risk assessment,Data-related risk,0
3,Inadequate risk control measures,Time-related risk,0
4,Neglecting maintenance work,0,1


In [ ]:
# Function to update values in the "types" and "risk" columns based on the condition
def update_types_risk(row):
    if not isinstance(row['types'], str):
        return row["types"], row["risk"]

    if row['types'].endswith("risk"):
        return row['risk'], row['types']
    else:
        return row['types'], row['risk']

# Apply the function to update the "types" and "risk" columns
val['types'], val['risk'] = zip(*val.apply(update_types_risk, axis=1))


In [105]:
val.to_csv("embedding_draft.csv", index=False)